In [1]:
import pandas as pd

In [60]:
inventario_bueno = pd.read_excel('input\\inventario_bueno.xlsx')
inventario_bueno = inventario_bueno.rename(columns = {'CODIGO': 'CODIGO ARTICULO', 'SERIE': 'NUMERO SERIE', 'LOTE': 'NUMERO LOTE', 'CANTIDAD': 'STOCK'})


movimientos_jueves = pd.read_excel('input\\movimientos_hasta_jueves.xlsx')
mask_salida = movimientos_jueves['TIPO MOVIMIENTO'] == 'SALIDA'
movimientos_jueves.loc[mask_salida, 'STOCK'] = (movimientos_jueves.loc[mask_salida, 'STOCK']) * -1


movimientos_viernes = pd.read_excel('input\\movimientos_hasta_viernes.xlsx')
mask_salida = movimientos_viernes['TIPO MOVIMIENTO'] == 'SALIDA'
movimientos_viernes.loc[mask_salida, 'STOCK'] = (movimientos_viernes.loc[mask_salida,'STOCK']) * -1

inventario_cargado_a_informatica = pd.read_excel('input\\INVENTARIO_09_09_2022.xlsx')
inventario_cargado_a_informatica = inventario_cargado_a_informatica.rename(columns = {'Codigo Articulo ': 'CODIGO ARTICULO', 'Lote': 'NUMERO LOTE', 'Numero Serie': 'NUMERO SERIE',
                                                                                      'Cantidad Movidad': 'STOCK'})

# Ahora, cada columna de CODIGO ARTICULO, NUMERO LOTE, NUMERO SERIE debe ir en str

def unificador_columnas(df):
    df['CODIGO ARTICULO'] = df['CODIGO ARTICULO'].astype(str)
    df['NUMERO SERIE'] = df['NUMERO SERIE'].astype(str)
    df['NUMERO LOTE'] = df['NUMERO LOTE'].astype(str)

    return df

inventario_bueno = unificador_columnas(inventario_bueno)
movimientos_jueves = unificador_columnas(movimientos_jueves)
movimientos_viernes = unificador_columnas(movimientos_viernes)
inventario_cargado_a_informatica = unificador_columnas(inventario_cargado_a_informatica)


C:\Users\gmolina\AppData\Local\Temp\ipykernel_8696\1659945005.py:5: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype=datetime64[ns])
  movimientos_jueves = pd.read_excel('input\\movimientos_hasta_jueves.xlsx')
C:\Users\gmolina\AppData\Local\Temp\ipykernel_8696\1659945005.py:10: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype=datetime64[ns])
  movimientos_viernes = pd.read_excel('input\\movimientos_hasta_viernes.xlsx')


# - Primero, quiero ver los artículos que entraron y salieron el viernes. Ya que, esa es la diferencia que hará con el archivo del jueves.

In [61]:
fecha_viernes = pd.to_datetime("09-09-22", dayfirst = True)
solo_viernes = movimientos_viernes.query('`FECHA MOVIMIENTO` >= @fecha_viernes')
solo_viernes


,FECHA MOVIMIENTO,TIPO MOVIMIENTO,FECHA VENCIMIENTO,NUMERO LOTE,NUMERO SERIE,PROVEEDOR,CODIGO ARTICULO,NOMBRE ARTICULO,STOCK
0,2022-09-09 12:59:04,SALIDA,2023-07-30 00:00:00,75RG1540,SINSERIE,999999999,F602000,TIAMINA 30 MILIGRAMOS 1 ML AMPOLLA,-100
1,2022-09-09 12:40:54,SALIDA,2025-11-19 00:00:00,22E20844,SINSERIE,999999999,I012000,TRANSDUCTOR DE PRESION SIMPLE,-40
2,2022-09-09 12:40:54,SALIDA,2027-02-22 00:00:00,22B0810JZX,SINSERIE,999999999,I358400,TUBO ENDOTRAQUEAL Nº 8.0,-10
3,2022-09-09 12:40:53,SALIDA,NaT,SL,SINSERIE,96556940,AS4727700,TOALLA CLINIC PAQUETE,-14
4,2022-09-09 12:40:52,SALIDA,2026-07-30 00:00:00,HM20210715,SINSERIE,999999999,I049000,TAPAS DE BRANULA,-500
...,...,...,...,...,...,...,...,...,...
319,2022-09-09 08:27:01,ENTRADA,2024-02-29 23:00:00,SL,691S09,91871000,B532700,TAPENTADOL 50 MG ACCION RAPIDA,600
320,2022-09-09 08:25:19,SALIDA,2023-12-22 00:00:00,2107,SINSERIE,76751494,B580200,FENTANILO 25 UGH PARCHE DERMICO,-40
321,2022-09-09 08:25:19,SALIDA,2024-09-30 00:00:00,2I2102939A,SINSERIE,76079782,M099200,EVEROLIMUS 10 M COMPRIMIDO,-60
322,2022-09-09 08:24:30,ENTRADA,2023-12-22 23:00:00,2107,SINSERIE,76751494,B580200,FENTANILO 25 UGH PARCHE DERMICO,40


Ahora, puede que hayan habido movimientos de entrada/salida que dieran 0. En esos artículos, el inventario con el archivo del jueves y el viernes hubiera sido el mismo. Por lo tanto, estos casos deben ser filtrados

In [62]:
movimientos_totales_solo_viernes = solo_viernes.groupby(by = ['CODIGO ARTICULO'])['STOCK'].sum()
movimientos_totales_solo_viernes

CODIGO ARTICULO
A031900      -170
A032200       -24
A950500       -10
A999700       -30
AL5900300       0
             ... 
S192800         0
S204700         0
S316800      -100
T100000         0
U230200     -1600
Name: STOCK, Length: 198, dtype: int64

Del resultado anterior, se puede ver que hubo movimiento de 198 artículos el día viernes

In [63]:
movimientos_totales_solo_viernes_sin_0 = movimientos_totales_solo_viernes[movimientos_totales_solo_viernes != 0]
print(movimientos_totales_solo_viernes_sin_0)

CODIGO ARTICULO
A031900      -170
A032200       -24
A950500       -10
A999700       -30
AL5909800      15
             ... 
S003500       -30
S009600      -240
S060100       -30
S316800      -100
U230200     -1600
Name: STOCK, Length: 156, dtype: int64


Con el resultado anterior, se identificaron los artículos QUE VAN a generar diferencias al hacer el inventario con el archivo del jueves y del viernes. En estos casos, hubieron 156 artículos que tuvieron un movimiento neto distinto a 0.

In [64]:
movimientos_totales_solo_viernes_sin_0.to_excel('codigos_diferencia_jueves_viernes.xlsx')

## Ahora, el mismo ejercicio, pero teniendo en cuenta el código/lote/serie.

In [65]:
movimientos_totales_solo_viernes = solo_viernes.groupby(by = ['CODIGO ARTICULO', 'NUMERO LOTE', 'NUMERO SERIE'])['STOCK'].sum()
movimientos_totales_solo_viernes


CODIGO ARTICULO  NUMERO LOTE  NUMERO SERIE
A031900          22052044     SINSERIE        -170
A032200          C12EO438AO   SINSERIE         -24
A950500          2201835      SINSERIE         -10
A999700          C12AOO08A0   SINSERIE         -15
                 C12F0233A0   SINSERIE         -15
                                              ... 
S204700          FNC028914G   SINSERIE           0
                 FNC028915G   SINSERIE           0
S316800          PC00017942   SINSERIE        -100
T100000          N0416B06     SINSERIE           0
U230200          202406LR     SINSERIE       -1600
Name: STOCK, Length: 215, dtype: int64

Hubo movimiento de 215 artículos en total.

In [66]:
movimientos_totales_solo_viernes_sin_0 = movimientos_totales_solo_viernes[movimientos_totales_solo_viernes != 0]
print(movimientos_totales_solo_viernes_sin_0)

CODIGO ARTICULO  NUMERO LOTE  NUMERO SERIE
A031900          22052044     SINSERIE        -170
A032200          C12EO438AO   SINSERIE         -24
A950500          2201835      SINSERIE         -10
A999700          C12AOO08A0   SINSERIE         -15
                 C12F0233A0   SINSERIE         -15
                                              ... 
S003500          22199X       SINSERIE         -30
S009600          49620788     SINSERIE        -240
S060100          MD21718904   SINSERIE         -30
S316800          PC00017942   SINSERIE        -100
U230200          202406LR     SINSERIE       -1600
Name: STOCK, Length: 168, dtype: int64


Hubo movimiento de 168 artículos con movimiento neto distinto a 0.

In [67]:
movimientos_totales_solo_viernes_sin_0.to_excel('Movimientos_distintos.xlsx')

# Entonces, ahora buscar las diferencias entre inventarios.

- En este caso, puede haber diferencias de código/lote/serie.
  - Debido a lo anterior, se hará un análisis de diferencias de código
  - Luego, un análisis de diferencia entre código/lote/serie.

# 23/09/2022

Ahora, quiero agregarle al inventario que enviamos (sólo con los movimientos hasta el jueves 08/09), los movimientos del viernes.

In [68]:
inventario_agrupado_informatica = inventario_cargado_a_informatica.groupby(by = ['CODIGO ARTICULO', 'NUMERO LOTE', 'NUMERO SERIE'])['STOCK'].sum()

In [69]:
inventario_que_debio_ser_enviado = inventario_agrupado_informatica.add(movimientos_totales_solo_viernes_sin_0, fill_value = 0)
inventario_que_debio_ser_enviado.to_excel('inventario_que_debio_ser_enviado.xlsx')